In [1]:
# Skriptas rpeaks sekų, gautų iš .json failo ir EKG įrašo, panaudojant NEUROKIT2, sulyginimui
# Nuskaitomi įrašų failai .npy 

from xmlrpc.client import SYSTEM_ERROR
import pandas as pd
import numpy as np
import json, shutil, sys
import neurokit2 as nk
from pathlib import Path
from vertinimas_util import AnalyseHeartrate


def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.5*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')
rec_dir_eval = Path(db_path, 'records_evaluated')

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
# file_path = Path(rec_dir,'info_create_z.json')
# with open(file_path) as json_file:
#     info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas


print("Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,")
print("nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant")
print("Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.")
print("Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/")
print("rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'")

# Įrašai, kur rpeak nesutapimas tik 1
list = ["1636423.936"]
list = ['1636490.526']
list = ['1636495.646']
list = ['1636499.394']
list = ['1632133.116']
list = ['1636563.972']
list = ['1636593.000']
list = ['1631141.137']
list = ['1631129.850']
list = ['1631082.735']
list = ['1631082.118']
list = ['1631081.543']
list = ['1637166.430']
list = ['1637178.685']
list = ['1633540.147']
list = ['1633958.174']
list = ['1630717.159']
list = ['1632123.567'] # 0
list = ['1632728.750'] # 10 Rpeaks, kurių Neurokit nesukuria, mediko sudėta, visi yra V, ML irgi pagauna
list = ['1643061.340'] # 10 rpeaks, ten, kur jų nėra, mediko panaikinta

list = ['1636451.235'] # Važinėjanti izolinija
list = ['1631066.094'] # Yra triukšmelis ties 9800. Nuėmiau rpeaks ties 49000 ir 50800, neaiškumas ties 75400
list = ['1637584.237'] # Pavirpėjimas prie 109200
list = ['1631047.940'] # 
list = ['1631039.807'] # 
list = ['1637584.866'] # 
list = ['1637585.491'] # 
list = ['1631027.284'] # 
list = ['1637619.150'] # Labai važinėjanti izolinija, kartais staigiai, raumenų triukšmų palyginus nedaug 
list = ['1631002.473'] # 

list = ['1633444.221'] # Yra izolinijos staigių pavaikščiojimų, raumenų triukšmų išlindimų. Dažnai N arba S ML atpažista, kaip V.
list = ['1630999.344'] # 0, Yra tik vienoje vietoje staigus izolinijos šoktelėjimas su triukšmų išlindimu ties 119000, ML V gaudo neblogai, yra dubletų, tripletų
list = ['1637628.331'] # 0, Nemažai S, ML juos neblogai gaudo

list = ['1637628.402'] # 0, Iššoka triukšmai tik ties 48000.
list = ['1637631.801'] # 0
list = ['1637189.804'] # 0, Reiktų parodyti gydytojui. išmestas rpeak 72858, nors man atrodo, kad jis ten yra, ir ten ekstrasistolė. Yra ir daugiau vietų, atrodo, kaip ekstrasistolės, tačiau nepažymėta. Reiktų parodyti gydytojui.
list = ['1636414.518'] # 0, Sutriukšmauja tik ties 81200 ir 119200
list = ['1634736.586'] # 1, Nestabili izolinija, kartais išlenda triukšmai
list = ['1632923.661'] # 1, Labai ilgas, > 4000. Praleistas pikas, gydytojas pažymėjo. Kokybė nėra aiški
list = ['1630844.567'] # 2

# 2023 01 13 trūko 1:30
list = ['1633443.596'] # 1, Gera kokybė suprastėja nuo 93000, daug S, ML nemažai S ir N priskiria V
list = ['1633434.823'] # 1, Buvo praleistas rpeaks, gydytojas įdėjo. Yra staigių izolinijos šokinėjimų.
list = ['1633434.198'] # 0
list = ['1633427.309'] # 2, Ties 30400 prasideda izolinijos  šokinėjimas
list = ['1633548.426'] # 0, Yra pašokinėjimų nuo 30000, bet nedaug
list = ['1633408.989'] # 0
list = ['1633408.480'] # 2, Yra gana didelės amplitudės raumenų triukšmų
list = ['1633396.439'] # 2
list = ['1634648.967'] # 0
list = ['1633395.185'] # 1
list = ['1633387.028'] # 1

# 2023 01 18 trūko 1:30
# Šiuose 4 failuose rpeaks visi neteisingi, dar sykį leidžiant Neurokit - rpeaks
# atstatomi į teisingas vietas. json sutvarkomas su skriptu 1_rpeaks_json_koregavimas.ipynb
list = ['1637624.073'] # Nesutampančių yra 1481, taisomas specialiomis priemonėmis
list = ['1637624.073_new'] # 1,  rpeaks pataisyti, Kokybė arti 0, 
# tačiau yra keleta fragmentų su ryškiais raumenų triukšmais. Rpeaks pataisyti su spec. skriptu
# list = ['1637621.569'] # nesutampančių 1354,  taisomas specialiomis priemonėmis
list = ['1637621.569_new'] # 0, pataisytas
# list = ['1637630.258'] # nesutampančių 1327,  taisomas specialiomis priemonėmis
list = ['1637630.258_new'] # 1, pataisytas
# list = ['1638807.019'] # nesutampančių 1192,  taisomas specialiomis priemonėmis
list = ['1638807.019_new'] # 0,  pataisytas

# list = ['1638818.942'] #0,  nesutampančių 93,  38695 rpeak pakoreguotas ne ten (atstačiau su Zive), 
# # likusieji visi surasti ties nedideliu piku tarp gretimų R, anotatoriaus panaikintų rpikų: 92, pridėtų: 0  
# Labai įdomus įrašas, reiktų gydytojo paaiškinimo: yra daug dvigubai didesnių RR intervalų.
list = ['1638818.315'] # 0,  nesutampančių 71, visi surasti ties nedideliu piku tarp gretimų R,
# anotatoriaus panaikintų rpikų: 71,, pridėtų: 0     
# Labai įdomus įrašas, reiktų gydytojo paaiškinimo: yra daug dvigubai didesnių RR intervalų.
list = ['1642628.030'] # 0,  nesutampančių 53, anotatoriaus panaikintų rpikų: 53, pridėtų: 0

# 2023 01 20 trūko 2:00
list = ['1642627.410'] # 0,  nesutampančių 41, anotatoriaus panaikintų rpikų: 41, pridėtų: 0    
list = ['1638801.784'] # 2 ,  nesutampančių 40, anotatoriaus panaikintų rpikų: 22, pridėtų: 18
list = ['1637623.546'] # 0 ,  nesutampančių 37, anotatoriaus panaikintų rpikų: 37, pridėtų: 0
#  - nesutampa N, ir po analizės - U
list = ['1637622.288'] # 1 ,  nesutampančių 33, anotatoriaus panaikintų rpikų: 32, pridėtų: 1,
#  uždėta ne vietoje - pataisiau rankomis. Kokybė arti 0, tačiau pradžioje yra didelis izolinijos lakstymas
list = ['1637624.172'] # 0 ,  nesutampančių 32, anotatoriaus panaikintų rpikų: 32, pridėtų: 0
list = ['1637624.799'] # 0 ,  nesutampančių 30, anotatoriaus panaikintų rpikų: 30, pridėtų: 0
list = ['1636596.137'] # 0 ,  nesutampančių 29, anotatoriaus panaikintų rpikų: 29, pridėtų: 0
list = ['1637622.914'] # 0 ,  nesutampančių 28, anotatoriaus panaikintų rpikų: 24, pridėtų: 4

# 2023 01 23 trūko X:00
list = ['1632252.683'] # 0 ,  nesutampančių 25, anotatoriaus panaikintų rpikų: 25, pridėtų: 0
# Trumpas triukšmelis ties 115750
list = ['1638817.688'] # 0 ,  nesutampančių 25, anotatoriaus panaikintų rpikų: 25, pridėtų: 0
# Yra nemažai pauzių
list = ['1640590.013'] # 2 ,  nesutampančių 24, anotatoriaus panaikintų rpikų: 22, pridėtų: 2
list = ['1643070.532'] # 1 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 20, pridėtų: 2
# Gana bjauri vieta ties 14300
list = ['1636596.765'] # 0 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 21, pridėtų: 1
list = ['1632731.258'] # 0 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 21, pridėtų: 1
list = ['1632729.377'] # 0 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 0, pridėtų: 19
# Kartais išlenda aiškiai matomi raumenų triukšmai, daug nesurastų rpeaks ties V, sužymėti ranka
list = ['1638803.024'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 11, pridėtų: 8
list = ['1638799.296'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 13, pridėtų: 6
list = ['1638799.296'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 13, pridėtų: 6
list = ['1643065.093'] # 0 ,  nesutampančių 18, anotatoriaus panaikintų rpikų: 18, pridėtų: 0
list = ['1642630.518'] # 0 ,  nesutampančių 18, anotatoriaus panaikintų rpikų: 18, pridėtų: 0
list = ['1637626.680'] # 0 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 16, pridėtų: 1
list = ['1632730.004'] # 0 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 0, pridėtų: 17
# Kartais išlenda nedidelis raumenų triukšmas, daug praleista rpeaks ties V, sužymėti ranka
list = ['1646711.162'] # 1 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 17, pridėtų: 0
# Labai šokinėjanti izolinija
list = ['1642635.482'] # 0 ,  nesutampančių 16, anotatoriaus panaikintų rpikų: 16, pridėtų: 0
# Gana stabilus įrašas, tačiau nuolat matosi nedidelis raumenų triukšmas

# 2023 01 26 trūko X:00
list = ['1643078.826'] # 1,  nesutampančių 16, anotatoriaus panaikintų rpikų: 16, pridėtų: 0
# Didžioji dalies kokybė gera, tik nedideli raumenų triukšmai, tačiau nuo 106000 prasideda 
# kontaktiniai ir raumenų triukšmai, izolinijos nestabilumas.
list = ['1637625.426'] # 0,  nesutampančių 15, anotatoriaus panaikintų rpikų: 14, pridėtų: 1
list = ['1632730.631'] # 0,  nesutampančių 15, anotatoriaus panaikintų rpikų: 0, pridėtų: 15
# Neurokit blogai gaudo rpikus ties V
list = ['1638806.132'] # 2,  nesutampančių 15, anotatoriaus panaikintų rpikų: 9, pridėtų: 6
# Nuo 80600 prasidėjo staigūs izolinijos šuoliai ir neaiškūs triukšmai
list = ['1631037.236'] # 2,  nesutampančių 13, anotatoriaus panaikintų rpikų: 5, pridėtų: 8
list = ['1638829.105'] # 1,  nesutampančių 13, anotatoriaus panaikintų rpikų: 10, pridėtų: 3
# Triukšmų nedaug, tik išlenda pačioje pabaigoje, tačiau labai vaikščioja izolinija
list = ['1643073.813'] # 1,  nesutampančių 13, anotatoriaus panaikintų rpikų: 13, pridėtų: 0
# Arčiau 0, bet yra keleta pašokinėjančių trumpų atkarpų.
list = ['1642633.618'] # 0,  nesutampančių 12, anotatoriaus panaikintų rpikų: 12, pridėtų: 0
# Kaip ir daug kur, neurokit suranda rpeaks, kur vizualiai visai nepanašu ("lygioje vietoje")
list = ['1643064.468'] # 0,  nesutampančių 11, anotatoriaus panaikintų rpikų: 11, pridėtų: 0
list = ['1642632.378'] # 0,  nesutampančių 11, anotatoriaus panaikintų rpikų: 11, pridėtų: 0
# Nėra labai švarus, tačiau priskyriau 0


# 2023 02 10 truko X:00
list = ['1638830.348'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Triukšmingas ir šokinėjanti izolinija, tačiau ir triukšmai nedideli ir šokinėjimo amplitudė
#  maža ir ne staigiai. Neurokit randa pikus ties beveik nematomais pikais
list = ['1625405.103'] # 1,  nesutampančių 10, anotatoriaus panaikintų rpikų: 3, pridėtų: 7
# izolinija vietomis  pašokinėja, nemažai izolinijos bangavimų, yra ir raumenų triukšmų
list = ['1642634.862'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Vidutinio intensyvumo raumenų triukšmas nuo 38600, 3-5 pikai 

# 2023 04 24 truko X:00
list = ['1642632.998'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Nedideli EMG ir BW



flag_eval_copy = True 

for file_name in list:
    # file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

    if (flag_eval_copy):
        # Perrašome failus į vertinamų failų aplanką
        src = Path(rec_dir, file_name)
        dst = Path(rec_dir_eval, file_name)

        # kopijuojame duomenis ir .json failus, išsaugodami metaduomenis su copy2
        shutil.copy2(src, dst)
        src = Path(rec_dir, file_name + '.json')
        dst = Path(rec_dir_eval, file_name + '.json')
        shutil.copy2(src, dst)
        print(f"\nFailai {file_name} ir {file_name+ '.json'} perrašyti  iš {rec_dir} į {rec_dir_eval}")
        print()

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # rpeaks_from_json = np.array([3,5,7,10,11,12,13,15,17,19,20,22,23,24])
    # print(rpeaks_from_json)

    print(f"rpeaks iš json failo: {len(rpeaks_from_json)}")
    # print(rpeaks_json) 

    # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'js'
    # Sulyginimui naudoju http://www.jsondiff.com/
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_json]
    # filename = file_name + '-js' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


        # II-a dalis: suformuojame rpeaks su Neurokitu,
        # pakartojant Zive skriptą iš analysis.py ir heartrate_analysis.py
    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    # signal = nk.signal_filter(signal=signal_raw, sampling_rate=200, lowcut=0.5, method="butterworth", order=5)
    signal = signal_raw    
    ecg_signal_df = pd.DataFrame(signal, columns=['orig'])

    analysis_results = AnalyseHeartrate(ecg_signal_df)
    rpeaks_from_signal = analysis_results['rpeaks']
    print(f"rpeaks iš signal: {len(rpeaks_from_signal)}")

    # print(rpeaks_from_signal)
    
  # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'sg'
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_signal]
    # filename = file_name + '-sg' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


            # Sulyginimas

    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    print("\nViso rankiniu būdu pridėtų rpikų:", len(ab))
    print("Reikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal")
    # https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html
    # print(ab)   # Return the unique values in a that are not in b
    flag = np.size(ab)
    if flag:
        sorter = np.argsort(rpeaks_from_json)
        idxs = sorter[np.searchsorted(rpeaks_from_json, ab, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ab[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')

    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    print("\nViso rankiniu būdu panaikintų rpikų:", len(ba))
    print("Reikšmės faile rpeaks_from_signal kurių nėra faile rpeaks_from_json")
    # print(ba)   # Return the unique values in b that are not in a
    flag = np.size(ba)
    if flag:
        sorter = np.argsort(rpeaks_from_signal)
        idxs = sorter[np.searchsorted(rpeaks_from_signal, ba, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ba[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')





OS in my system :  linux
Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,
nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant
Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.
Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/
rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'

Zive įrašas:  1642632.998

Failai 1642632.998 ir 1642632.998.json perrašyti  iš /home/kesju/DI/DUOM_2022_RUDUO_2/records_selected į /home/kesju/DI/DUOM_2022_RUDUO_2/records_evaluated

rpeaks iš json failo: 544
rpeaks iš signal: 554

Viso rankiniu būdu pridėtų rpikų: 0
Reikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal
Tokių reikšmių nėra

Viso rankiniu būdu panaikintų rpikų: 10
Reikšmės faile rpeaks_from_signal kurių nėra faile rpeaks_from_json
i     index   rpeak
0        50   11928
1        64   15085
2        68   15827
3        72   16570
4

In [2]:
for file_name in list:
    file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # print(rpeaks_from_json[:5])

            # II-a dalis: suformuojame rpeaks su Neurokitu

    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    # print(rpeaks_from_signal[:5])
    
  
            # Sulyginimas

    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    print(f"Nesutampančių rpeaks skaičius NS: {np.size(ab)+np.size(ba)} ")





Zive įrašas:  1642632.998
Nesutampančių rpeaks skaičius NS: 10 
